In [30]:
import sys
sys.path.append('..')

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

In [47]:
df = pd.read_csv('../data/forex_factory_data.csv')

In [48]:
df.shape

(24072, 8)

In [49]:
df.head()

,date,time,currency,impact,event,actual_value,forecast_value,prev_value
0,Wed - Jan 1 - 2020,All Day,NZD,Non-Economic,Bank Holiday,NaN,NaN,NaN
1,Wed - Jan 1 - 2020,All Day,JPY,Non-Economic,Bank Holiday,NaN,NaN,NaN
2,Wed - Jan 1 - 2020,All Day,CNY,Non-Economic,Bank Holiday,NaN,NaN,NaN
3,Wed - Jan 1 - 2020,All Day,CHF,Non-Economic,Bank Holiday,NaN,NaN,NaN
4,Wed - Jan 1 - 2020,All Day,EUR,Non-Economic,French Bank Holiday,NaN,NaN,NaN


In [50]:
df[['date']].head()

,date
0,Wed - Jan 1 - 2020
1,Wed - Jan 1 - 2020
2,Wed - Jan 1 - 2020
3,Wed - Jan 1 - 2020
4,Wed - Jan 1 - 2020


In [51]:
# Now I want to make 2 more columns from this for day and date from date column. 
date = 'Wed - Jan 1 - 2020'
date.split(' - ')

['Wed', 'Jan 1', '2020']

In [52]:
df['day'] = df['date'].apply(lambda x: x.split(' - ')[0])
df['datee'] = df['date'].apply(lambda x: ' '.join(x.split(' - ')[1:]))

In [53]:
df.drop(columns=['date'], inplace=True)
df.rename(columns={'datee': 'date'}, inplace=True)
df.head()

,time,currency,impact,event,actual_value,forecast_value,prev_value,day,date
0,All Day,NZD,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,Jan 1 2020
1,All Day,JPY,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,Jan 1 2020
2,All Day,CNY,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,Jan 1 2020
3,All Day,CHF,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,Jan 1 2020
4,All Day,EUR,Non-Economic,French Bank Holiday,NaN,NaN,NaN,Wed,Jan 1 2020


In [54]:
# Function to clean and parse the time
def clean_time(time_str):
    try:
        # Try parsing the time using datetime for valid time values
        return datetime.strptime(time_str.strip().lower(), '%I:%M%p').time()
    except:
        # Return None for invalid or missing values
        return None


In [55]:
df['clean_time'] = df['time'].apply(clean_time)
df['date'] = df['date'].str.strip()

In [64]:
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')

In [ ]:
\

,time,currency,impact,event,actual_value,forecast_value,prev_value,day,date,clean_time,datetime,datetime_iso
24067,6:30am,CNY,High,Manufacturing PMI,NaN,NaN,NaN,Tue,2024-12-31,06:30:00,None,None
24068,6:30am,CNY,Medium,Non-Manufacturing PMI,NaN,NaN,NaN,Tue,2024-12-31,06:30:00,None,None
24069,All Day,EUR,Non-Economic,German Bank Holiday,NaN,NaN,NaN,Tue,2024-12-31,None,None,None
24070,7:00pm,USD,Medium,S&P/CS Composite-20 HPI y/y,NaN,NaN,NaN,Tue,2024-12-31,19:00:00,None,None
24071,7:00pm,USD,Low,HPI m/m,NaN,NaN,NaN,Tue,2024-12-31,19:00:00,None,None


In [67]:
def combine_datetime(row):
    try:
        # If clean_time is valid, append it to the date
        time = pd.to_datetime(row['clean_time'], format='%H:%M:%S', errors='coerce').time()
        if pd.isnull(time):
            raise ValueError
    except:
        # Default to '00:00:00' if invalid or missing
        time = '00:00:00'
    return f"{row['date']} {time}"

In [68]:
df['datetime'] = df.apply(combine_datetime, axis=1)

In [70]:
df.tail()

,time,currency,impact,event,actual_value,forecast_value,prev_value,day,date,clean_time,datetime,datetime_iso
24067,6:30am,CNY,High,Manufacturing PMI,NaN,NaN,NaN,Tue,2024-12-31,06:30:00,2024-12-31 06:30:00,None
24068,6:30am,CNY,Medium,Non-Manufacturing PMI,NaN,NaN,NaN,Tue,2024-12-31,06:30:00,2024-12-31 06:30:00,None
24069,All Day,EUR,Non-Economic,German Bank Holiday,NaN,NaN,NaN,Tue,2024-12-31,None,2024-12-31 00:00:00,None
24070,7:00pm,USD,Medium,S&P/CS Composite-20 HPI y/y,NaN,NaN,NaN,Tue,2024-12-31,19:00:00,2024-12-31 19:00:00,None
24071,7:00pm,USD,Low,HPI m/m,NaN,NaN,NaN,Tue,2024-12-31,19:00:00,2024-12-31 19:00:00,None


In [71]:
df.drop(columns=['datetime_iso', 'clean_time'], inplace=True)

In [74]:
df.head()

,time,currency,impact,event,actual_value,forecast_value,prev_value,day,date,datetime
0,All Day,NZD,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,2020-01-01,2020-01-01 00:00:00
1,All Day,JPY,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,2020-01-01,2020-01-01 00:00:00
2,All Day,CNY,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,2020-01-01,2020-01-01 00:00:00
3,All Day,CHF,Non-Economic,Bank Holiday,NaN,NaN,NaN,Wed,2020-01-01,2020-01-01 00:00:00
4,All Day,EUR,Non-Economic,French Bank Holiday,NaN,NaN,NaN,Wed,2020-01-01,2020-01-01 00:00:00


In [76]:
df.to_csv('../data/forex_factory_data_cleaned.csv', index=False)